In [19]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import pandas as pd
import csv

In [20]:
# TODO: need to prompt the user to input email
USER_EMAIL = "lbo@multusmanagement.com"
PASSWORD = "B3lieveSt0ne!"

# Account: Stephanie
# USER_EMAIL = "stephanie.francor@gmail.com"
# PASSWORD = "wDl37N16v$^a"

# Account: DSIS
# USER_EMAIL = "dsis@multusmanagement.com"
# PASSWORD = "B3lieveSt0ne!"

# Account: BHI
# USER_EMAIL = "BHI2@multusmanagement.com"
# PASSWORD = "5Az*32MjEW4p"

# Account: Alpine
# USER_EMAIL = "alpine@multusmanagement.com"
# PASSWORD = "B3lieveSt0ne!"

# Account: PT300
# USER_EMAIL = "pt300@multusmanagement.com"
# PASSWORD = "B3lieveSt0ne!"

MAIN_PAGE_URL = "https://www.airbnb.com/login"
LISTING_PAGE_URL = "https://www.airbnb.com/hosting/listings"
RESERVATION_URL = "https://www.airbnb.com/hosting/reservations"

XPATH_email_btn = "/html/body/div[5]/div/div/div[1]/div/div[2]/div[1]/main/div/div/div/div/div/div[3]/div/div[4]/button/div/div[2]"
XPATH_login_form = "/html/body/div[5]/div/div/div[1]/div/div[2]/div[1]/main/div/div/div/div/div/form"
XPATH_completed_btn = "/html/body/div[5]/div/div/div[1]/div/div/main/div[1]/section/div[4]/div[1]/div/div/div/div/div[1]/div/button[2]"
XPATH_canceled_btn = "/html/body/div[5]/div/div/div[1]/div/div/main/div[1]/section/div[4]/div[1]/div/div/div/div/div[1]/div/button[3]"
XPATH_reservation_next_page_btn = "/html/body/div[5]/div/div/div[1]/div/div/main/div[1]/section/footer/div/nav/div/button[8]"
CSS_SELECTOR_next_btn = "#site-content > div._1u2ubxs > section > footer > div > nav > div > button._1bfat5l.l1j9v1wn.dir.dir-ltr"
                        # "#site-content > div._1u2ubxs > section > footer > div > nav > div > button._1bfat5l.l1j9v1wn.dir.dir-ltr"
                        # "#site-content > div._1u2ubxs > section > footer > div > nav > div > button._1bfat5l.l1j9v1wn.dir.dir-ltr"


In [21]:
driver = webdriver.Chrome()
driver.get(MAIN_PAGE_URL)

In [22]:
def login(driver):
    # click continue with email with XPATH - doable
    email_btn = driver.find_element(By.XPATH, XPATH_email_btn)
    email_btn.click() 

    # try input user info - workable
    email_input = driver.find_element(By.NAME, "user[email]")
    email_input.send_keys(USER_EMAIL)

    # find "continue" button and click to submit email
    login_form = driver.find_element(By.XPATH, XPATH_login_form)
    login_form.submit()
    # TODO: after submitting, not shown (might need to sleep)
    time.sleep(5)

    # find password input form and input password
    pw_input = driver.find_element(By.NAME, "user[password]")
    pw_input.send_keys(PASSWORD)
    pw_input.submit()

    # TODO: SUCCESSFULLY LOGGED IN
    time.sleep(5)


In [23]:
login(driver)
# SMS code 

In [24]:
driver.get(RESERVATION_URL)

In [25]:
def extract_all_entries(html_source):
    all_entries = html_source.split("<tr><td class=\"_qjegjiv\">")[1:]
    all_entries[0] = "<tr><td class=\"_qjegjiv\">" + all_entries[0]
    all_entries[-1] = all_entries[-1].split("</div></td></tr>")[0]
    return all_entries

In [26]:
def append_one_entry(df, entry):
    splitted_by_status = entry.split("<div class=\"_pgfqnw\"><span>")
    status = splitted_by_status[1].split("</span>")[0]
    splitted_by_others = splitted_by_status[1].split("<td class=\"_qjegjiv\">")
    
    check_in_time = splitted_by_others[1].split("</td>")[0]
    check_out_time = splitted_by_others[2].split("</td>")[0]
    listing_name = splitted_by_others[3].split("</td>")[1].split(">")[1]
    room = listing_name.split("*")[-1]
    confirmation_code = splitted_by_others[4].split("</td>")[0]
    total_payout = splitted_by_others[5].split("</td>")[0].split("$")[1]

    df["listing name"].append(listing_name)
    df["room"].append(room)
    df["confirmation"].append(confirmation_code)
    df["status"].append(status)
    df["check in"].append(check_in_time)
    df["check out"].append(check_out_time)
    df["total payout"].append(total_payout)
    # TODO: internal name
    
def append_one_entry_completed(df, entry):
    splitted_by_status = entry.split("<div class=\"_pgfqnw\"><span")
    status = splitted_by_status[1].split("</span>")[0][18:]
    splitted_by_others = splitted_by_status[1].split("<td class=\"_qjegjiv\">")
    
    check_in_time = splitted_by_others[1].split("</td>")[0]
    check_out_time = splitted_by_others[2].split("</td>")[0]
    listing_name = splitted_by_others[3].split("</td>")[1].split(">")[1]
    room = listing_name.split("*")[-1]
    confirmation_code = splitted_by_others[4].split("</td>")[0]
    total_payout = splitted_by_others[5].split("</td>")[0].split("$")[1]

    df["listing name"].append(listing_name)
    df["room"].append(room)
    df["confirmation"].append(confirmation_code)
    df["status"].append(status)
    df["check in"].append(check_in_time)
    df["check out"].append(check_out_time)
    df["total payout"].append(total_payout)
    # TODO: internal name
    
def append_all_entries(df, html_source):
    all_entries = extract_all_entries(html_source)
    for entry in all_entries:
        # print(entry)
        append_one_entry(df, entry)

def append_all_entries_completed(df, html_source):
    all_entries = extract_all_entries(html_source)
    for entry in all_entries:
        # print(entry)
        append_one_entry_completed(df, entry)
    
def append_all_pages_upcoming(driver, df):
    html_source = ""
    while (html_source != driver.page_source):
        html_source = driver.page_source
        append_all_entries(df, html_source)
        # next_page_btn = driver.find_element(By.XPATH, XPATH_reservation_next_page_btn)
        potential_elements = driver.find_elements(By.CSS_SELECTOR, CSS_SELECTOR_next_btn)
        if len(potential_elements) == 0:
            print("no next page")
            # break
        else:
            next_page_btn = potential_elements[0]
            next_page_btn.click()
        time.sleep(3.5)

def append_all_pages_completed(driver, df):
    html_source = ""
    # counter = 0
    while (html_source != driver.page_source):
        html_source = driver.page_source
        append_all_entries_completed(df, html_source)
        # next_page_btn = driver.find_element(By.XPATH, XPATH_reservation_next_page_btn)
        potential_elements = driver.find_elements(By.CSS_SELECTOR, CSS_SELECTOR_next_btn)
        if len(potential_elements) == 0:
            print("no next page")
            # break
        else:
            next_page_btn = potential_elements[0]
            next_page_btn.click()
        time.sleep(5)
        # counter = counter + 1

def append_all_pages_canceled_test(driver, df):
    html_source = ""
    while (html_source != driver.page_source):
        html_source = driver.page_source
        append_all_entries(df, html_source)
        # next_page_btn = driver.find_element(By.XPATH, XPATH_reservation_next_page_btn)
        potential_elements = driver.find_elements(By.CSS_SELECTOR, CSS_SELECTOR_next_btn)
        if len(potential_elements) == 0:
            break
        else:
            next_page_btn = potential_elements[0]
            next_page_btn.click()
        time.sleep(4)



In [27]:
df_dict_upcoming = {
    "listing name": [],
    "room": [],
    "confirmation": [],
    "status": [],
    "check in": [],
    "check out": [],
    "total payout": []
}
# append_all_entries_completed(df_dict_upcoming, driver.page_source)
append_all_pages_upcoming(driver, df_dict_upcoming)
df_upcoming = pd.DataFrame(df_dict_upcoming)
df_upcoming

no next page


,listing name,room,confirmation,status,check in,check out,total payout
0,BeachFront*1 King*WIFI*Security*Laundry*Room-103,Room-103,HMWWKF22R8,Currently hosting,"Mar 23, 2023","Mar 24, 2023",88.75
1,BeachFront*2 Full*WIFI*Security*Laundry*Room-106,Room-106,HMC84D59BM,Currently hosting,"Mar 23, 2023","Mar 25, 2023",193.57
2,BeachFront*2 full*WIFI*Security*Laundry*Room-125,Room-125,HMY25AJ2RQ,Currently hosting,"Mar 23, 2023","Mar 26, 2023",297.48
3,BeachFront*1 King*WIFI*Security*Laundry*Room-103,Room-103,HMYPJZ2ZCM,Arriving tomorrow,"Mar 24, 2023","Mar 25, 2023",96.03
4,Ocean View*BeachFront*2 Full*WIFI*Laundry*Room...,Room 108,HMT58FZRZY,Arriving tomorrow,"Mar 24, 2023","Mar 26, 2023",200.10
5,BeachFront*1 King*WIFI*Security*Laundry*Room-103,Room-103,HMZMT59ZA5,Arriving in 2 days,"Mar 25, 2023","Mar 26, 2023",102.49
6,Ocean View*BeachFront*2 Full*WIFI*Laundry*Room...,Room 108,HM4FJYWEPP,Confirmed,"May 17, 2023","May 22, 2023",895.89


In [15]:
# df_upcoming.to_csv("./stephanie-reservation-upcoming.csv", index=False)
# df_upcoming.to_csv("./03-20-reservations/DSIS-reservation-upcoming.csv", index=False)
# df_upcoming.to_csv("./BHI-reservation-upcoming.csv", index=False)
df_upcoming.to_csv("./03-20-reservations/LBO-reservation-upcoming.csv", index=False)
# df_upcoming.to_csv("./PT300-reservation-upcoming.csv", index=False)

In [101]:
# df.to_csv("./out.csv", index=False)
# df["xxxx"] == "arriving ......"

In [11]:
# getting completed reservations
completed_bnt = driver.find_element(By.XPATH, XPATH_completed_btn)
completed_bnt.click()

In [12]:
df_dict_completed = {
    "listing name": [],
    "room": [],
    "confirmation": [],
    "status": [],
    "check in": [],
    "check out": [],
    "total payout": []
}

append_all_pages_completed(driver, df_dict_completed)
df_completed = pd.DataFrame(df_dict_completed)
df_completed

,listing name,room,confirmation,status,check in,check out,total payout
0,BeachFront*2 Full*WIFI*Security*Laundry*Room-126,Room-126,HM3ZE5YANM,Review guest,"Mar 18, 2023","Mar 19, 2023",126.10
1,BeachFront*2 Full*WIFI*Security*Laundry*Room-131,Room-131,HMB5PCZBFR,Review guest,"Mar 18, 2023","Mar 19, 2023",126.10
2,BeachFront*2 Full*WIFI*Security*Laundry*Room-106,Room-106,HMYTTYR9BS,Review guest,"Mar 18, 2023","Mar 19, 2023",85.36
3,BeachFront*2 full*WIFI*Security*Laundry*Room-127,Room-127,HMRJEWAHAH,Review guest,"Mar 17, 2023","Mar 19, 2023",213.40
4,Ocean View*BeachFront*2 Full*WIFI*Laundry*Room...,Room 108,HMX4F9MS28,Review guest,"Mar 18, 2023","Mar 19, 2023",126.10
...,...,...,...,...,...,...,...
352,BeachFront*1 King 2 full*Security*Laundry*Room...,Room-123,HMQSEFNKZ5,Past guest,"Jan 13, 2023","Jan 14, 2023",48.50
353,BeachFront*2 Full*WIFI*Security*Laundry*Room-109,Room-109,HM3YRMDQW4,Past guest,"Jan 13, 2023","Jan 14, 2023",48.50
354,BeachFront*2 Full*WIFI*Security*Laundry*Room-106,Room-106,HM5234W5W2,Past guest,"Jan 13, 2023","Jan 14, 2023",48.50
355,Discard-108,Discard-108,HMJYB4ZM53,Past guest,"Jan 12, 2023","Jan 14, 2023",87.30


In [18]:
# df_completed.to_csv("./stephanie-reservation-completed.csv", index=False)
# df_completed.to_csv("./03-20-reservations/DSIS-reservation-completed.csv", index=False)
# df_completed.to_csv("./BHI-reservation-completed.csv", index=False)
df_completed.to_csv("./03-20-reservations/LBO-reservation-completed.csv", index=False)
# df_completed.to_csv("./PT300-reservation-completed.csv", index=False)

In [14]:
# getting canceled reservations
canceled_bnt = driver.find_element(By.XPATH, XPATH_canceled_btn)
canceled_bnt.click()

In [16]:
df_dict_canceled = {
    "listing name": [],
    "room": [],
    "confirmation": [],
    "status": [],
    "check in": [],
    "check out": [],
    "total payout": []
}

append_all_pages_canceled_test(driver, df_dict_canceled)
df_canceled = pd.DataFrame(df_dict_canceled)
df_canceled

,listing name,room,confirmation,status,check in,check out,total payout
0,BeachFront*2 Full*WIFI*Security*Laundry*Room-117,Room-117,HM2RJ9NCHQ,Canceled by guest,"Mar 17, 2023","Mar 18, 2023",0.00
1,BeachFront*2 Full*WIFI*Security*Laundry*Room-109,Room-109,HMXPFN8JWK,Canceled by guest,"Mar 10, 2023","Mar 16, 2023",641.65
2,BeachFront*2 full*WIFI*Security*Laundry*Room-122,Room-122,HMWBJ4T2CD,Canceled by guest,"Mar 10, 2023","Mar 12, 2023",0.00
3,BeachFront*1 King*WIFI*Security*Laundry*Room-136,Room-136,HMEDBQEA9E,Canceled by guest,"Mar 10, 2023","Mar 12, 2023",0.00
4,BeachFront*1 King*WIFI*Security*Laundry*Room-137,Room-137,HMJDYRD2MB,Canceled by guest,"Mar 10, 2023","Mar 12, 2023",363.75
5,BeachFront*2 Full*WIFI*Security*Laundry*Room- 110,Room- 110,HMN3H9NBD8,Canceled by Airbnb,"Mar 10, 2023","Mar 12, 2023",0.00
6,Discard-108,Discard-108,HMBSY43ZDK,Canceled by guest,"Mar 10, 2023","Mar 12, 2023",0.00
7,BeachFront*1 King*WIFI*Security*Laundry*Room-136,Room-136,HMX9N8THNE,Canceled by Airbnb,"Mar 10, 2023","Mar 11, 2023",0.00
8,BeachFront*2 full*WIFI*Security*Laundry*Room-118,Room-118,HMRHFM3SH9,Canceled by guest,"Mar 9, 2023","Mar 12, 2023",0.00
9,BeachFront*2 Full*WIFI*Security*Laundry*Room-112,Room-112,HMRFKK5YHH,Canceled by guest,"Mar 9, 2023","Mar 12, 2023",612.31


In [17]:
# df_canceled.to_csv("./stephanie-reservation-canceled.csv", index=False)
# df_canceled.to_csv("./03-20-reservations/DSIS-reservation-canceled.csv", index=False)
# df_canceled.to_csv("./BHI-reservation-canceled.csv", index=False)
df_canceled.to_csv("./03-20-reservations/LBO-reservation-canceled.csv", index=False)
# df_canceled.to_csv("./PT300-reservation-canceled.csv", index=False)

In [ ]:
# internal name XPATH
# /html/body/div[6]/div/div/div[1]/div/div/main/div/div/div/div/section[1]/div/div/div[2]/div[1]/table/tbody[2]/tr[1]/td[2]/div/div[2]/div[2]